In [1]:
import networkx as nx
import random
import math
import matplotlib
import pickle
from matplotlib import pyplot as plt
from random import shuffle
from copy import deepcopy
import numpy as np
import time
import sys
#from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [2]:
print(nx.__version__)
print(matplotlib.__version__)

2.1
2.1.2


In [6]:
num_of_nodes = 100

# Функции для генерации случайных графов

In [3]:
def get_ab(alpha1, alpha2, betta1, betta1_packet):
    #print(alpha1, betta1, betta1_packet)
    random.seed()
    if alpha1 == -1000:
        alpha1 = random.random() * math.pi * 0.5
    
    random.seed()
    if alpha2 == -1000:
        alpha2 = random.random() * math.pi * 0.5
    
    x = random.randint(0, 1)
    if betta1 == -1000:
        #betta1 = random.randint(-(num_of_nodes-1), (num_of_nodes-1))
        betta_temp = random.randint(1, (num_of_nodes-1))
        if x == 0:
            betta1 = betta_temp
        else:
            betta1 = - betta_temp * math.tan(alpha1)
    else:
        betta_temp = betta1_packet[0]
        x = betta1_packet[1]
        if x == 1:
            betta1 = - betta_temp * math.tan(alpha1)
        else:
            betta1 = betta_temp
        
    #time.sleep(0.005)
    
    return alpha1, betta1, alpha2, betta_temp, x

In [4]:
def swap(arr, i):
    while 1==1:
        j = random.randint(0, len(arr)-1)
        if j != i:
            k = arr[i]
            arr[i] = arr[j]
            arr[j] = k
            break
    #return arr

In [5]:
def get_list_of_values(alpha, betta, x0, xN, num_of_nodes):
    f_values = []
    x_min = max([0, x0]) # max([0, x0])
    x_max = min([num_of_nodes, xN]) # min([num_of_nodes, xN])
    for i in range(0, num_of_nodes):
        f = i * math.tan(alpha) + betta
        if i < x_min:
            f_values.append(1) # 0
        elif i > x_max:
            f_values.append(num_of_nodes - 1) # num_of_nodes
        else:
            f_values.append(int(f))
    return f_values
        

def get_area(alpha, betta, num_of_nodes):
    x0 = (1 - betta) / math.tan(alpha)
    xN = (num_of_nodes - 1 - betta) / math.tan(alpha)
    #print(x0, xN)
    f_val = get_list_of_values(alpha, betta, x0, xN, num_of_nodes)
    s = 0
    for i in range(0, num_of_nodes):
        s = s + f_val[i]
    return s, f_val
    """
    if (betta >= 0) and (xN > num_of_nodes):
        return (betta * num_of_nodes + (num_of_nodes ** 2) * math.tan(alpha)), get_list_of_values(alpha, betta, x0, xN, num_of_nodes)
    elif (betta >= 0) and (xN <= num_of_nodes):
        return (num_of_nodes ** 2 - 0.5 * num_of_nodes * xN + 0.5 * betta), get_list_of_values(alpha, betta, x0, xN, num_of_nodes)
    elif (betta < 0) and (xN > num_of_nodes):
        return (0.5 * (num_of_nodes - x0) * (num_of_nodes * math.tan(alpha) + betta)), get_list_of_values(alpha, betta, x0, xN, num_of_nodes)
    elif (betta < 0) and (xN <= num_of_nodes):
        return (0.5 * ((num_of_nodes - x0) + (num_of_nodes - xN)) * num_of_nodes), get_list_of_values(alpha, betta, x0, xN, num_of_nodes)
    """    

In [8]:
# Generate dict for prediction of degree distributions

def get_predicted(num_of_nodes, alpha1=-1000, alpha2=-1000, betta1=-1000, betta1_packet = []):
    alpha1_initial = alpha1
    alpha2_initial = alpha2
    betta1_initial = betta1
    betta1_packet_initial = betta1_packet
    while 1==1:
        alpha1, betta1, alpha2, betta_temp, x = get_ab(alpha1_initial, alpha2_initial, betta1_initial, betta1_packet_initial)
        s1, f_val1 = get_area(alpha1, betta1, num_of_nodes)

        betta2 = 0
        s2 = 0
        f_val2 = []
        min_diff = num_of_nodes * num_of_nodes
        #for _betta2 in range(-num_of_nodes+1, num_of_nodes):

        finish = False
        for _betta2 in range(1, num_of_nodes):
            _s2, _f_val2 = get_area(alpha2, _betta2, num_of_nodes)
            if min_diff > abs(_s2 - s1):
                min_diff = abs(_s2 - s1)
                betta2 = _betta2
                s2 = _s2
                f_val2 = _f_val2
            #if (s2 != 0) and (s1 != 0):
            #    if (abs(s2 - s1) / s2) <= 0.05:
            #        finish = True
            #        break

        ##if not finish:
        for _betta2_temp in range(1, num_of_nodes):
            _betta2 = - _betta2_temp * math.tan(alpha2)
            _s2, _f_val2 = get_area(alpha2, _betta2, num_of_nodes)
            if min_diff > abs(_s2 - s1):
                min_diff = abs(_s2 - s1)
                betta2 = _betta2
                s2 = _s2
                f_val2 = _f_val2
        if (s2 != 0) and (s1 != 0):
            if (abs(s2 - s1) / s2) <= 0.05:
                finish = True
                #break
        
        if finish:
            break
    
    return {'alpha1':alpha1, 'betta1':betta1, 'alpha2':alpha2, 'betta2':betta2,\
            'betta_temp':betta_temp, 'x':x,\
            'f_val1':f_val1, 'f_val2':f_val2}

In [9]:
def fill_edges_array(f_val):
    edges_array = []
    
    elements = list(range(len(f_val)))
    shuffle(elements)
    
    for i in range(len(f_val)):
        for j in range(f_val[i]):
            edges_array.append(elements[i]) # i
    return edges_array

"""
def outgoing_ingoing_random_swap(arr1, arr2):
    outgoing = deepcopy(arr1)
    ingoing = deepcopy(arr2)
    swap = random.randint(0,1)
    if swap == 0:
        return outgoing, ingoing
    else:
        return ingoing, outgoing
"""
def outgoing_ingoing_random_swap(predicted_dict):
    swap = random.randint(0,1)
    if swap == 1:
        fval1 = deepcopy(predicted_dict['f_val1'])
        fval2 = deepcopy(predicted_dict['f_val2'])
        predicted_dict['f_val1'] = fval2
        predicted_dict['f_val2'] = fval1
        alpha1 = predicted_dict['alpha1']
        alpha2 = predicted_dict['alpha2']
        betta1 = predicted_dict['betta1']
        betta2 = predicted_dict['betta2']
        predicted_dict['alpha1'] = alpha2
        predicted_dict['alpha2'] = alpha1
        predicted_dict['betta1'] = betta2
        predicted_dict['betta2'] = betta1
        return 1
    else:
        return 0

In [10]:
# Generate graph using prediction dict

def get_fact(predicted_dict, num_of_nodes):
    G = nx.connected_watts_strogatz_graph(num_of_nodes, 5, 0.1)
    #G_mst = nx.algorithms.tree.minimum_spanning_tree(G)
    #G = G_mst

    edgelist = list(G.edges())
    G = nx.DiGraph()
    #G.add_edges_from(edgelist)
    
    random_swap = outgoing_ingoing_random_swap(predicted_dict)

    outgoing = fill_edges_array(predicted_dict['f_val1'])
    ingoing = fill_edges_array(predicted_dict['f_val2'])
    
    #outgoing, ingoing = outgoing_ingoing_random_swap(outgoing, ingoing)

    #num_of_edges_to_add = min([s1,s2]) # - G.number_of_edges()
    edge_counter = 0
    iter_counter = 0
    """
    while 1 == 1:
        u = outgoing[random.randint(0, len(outgoing)-1)]
        v = ingoing[random.randint(0, len(ingoing)-1)]
        if not G.has_edge(u, v):
            G.add_edge(u, v)
            edge_counter = edge_counter + 1
        if edge_counter == num_of_edges_to_add:
            print('Ok')
            break
        if iter_counter == num_of_edges_to_add * 10:
            print('Fail')
            break
    """

    shuffle(outgoing)
    shuffle(ingoing)
    counter = 0
    #for i in range(max(len(outgoing), len(ingoing))):
    i = 0
    while i < max(len(outgoing), len(ingoing)):
        pos_out = i % len(outgoing)
        pos_in = i % len(ingoing)
        u = outgoing[pos_out]
        v = ingoing[pos_in]
        #print(u,v)
        if (not G.has_edge(u, v)) and (u != v):
        #if not G.has_edge(u, v):
            G.add_edge(u, v)
        else:
            #shuffle(outgoing)
            #shuffle(ingoing)
            swap(outgoing, pos_out)
            swap(ingoing, pos_in)
            i = i - 1
            counter = counter + 1
        i = i + 1
    #print(counter)
    #"""
    
    outgoing_deg = list(dict(G.out_degree()).values())
    ingoing_deg = list(dict(G.in_degree()).values())
    outgoing_deg.sort()
    ingoing_deg.sort()
    
    outgoing_error_list = []
    ingoing_error_list = []
    for i in range(len(outgoing_deg)):
        outgoing_error_list.append(abs(predicted_dict['f_val1'][i] - outgoing_deg[i]))
        ingoing_error_list.append(abs(predicted_dict['f_val2'][i] - ingoing_deg[i]))
    outgoing_error = 0
    ingoing_error = 0
    for i in range(len(outgoing_error_list)):
        outgoing_error = outgoing_error + outgoing_error_list[i]
        ingoing_error = ingoing_error + ingoing_error_list[i]
    
    return {'G':G, 'outgoing_deg':outgoing_deg, 'ingoing_deg':ingoing_deg,\
            'swap_counter':counter, 'outgoing_error':outgoing_error, 'ingoing_error':ingoing_error,\
           'num_of_edges':G.number_of_edges(),\
           'random_swap':random_swap}

# Демонстрация

In [12]:
# Some tests just to see how it works

num_of_nodes = 1000
predicted_dict = get_predicted(num_of_nodes,\
                               alpha1=(math.pi/4-math.pi/8),alpha2=(math.pi/4+math.pi/8),\
                               betta1=0, betta1_packet=[0,0])
fact_dict = get_fact(predicted_dict, num_of_nodes)

print('alpha1:', predicted_dict['alpha1'])
print('alpha2:', predicted_dict['alpha2'])
print('betta1:', predicted_dict['betta1'])
print('betta_temp:', predicted_dict['betta_temp'])
print('x:', predicted_dict['x'])
print('alpha2:', predicted_dict['alpha2'])
print('betta2:', predicted_dict['betta2'])
print()
print('random_swap:', fact_dict['random_swap'])
print('swap_counter:', fact_dict['swap_counter'])
print('outgoing_error:', fact_dict['outgoing_error'])
print('ingoing_error:', fact_dict['ingoing_error'])
print()
print('f_val1:', sum(predicted_dict['f_val1']))
print('f_val2:', sum(predicted_dict['f_val2']))
print('number of edges:', fact_dict['G'].number_of_edges())

#plt.xlim((0, num_of_nodes))
#plt.ylim((0, num_of_nodes))
#plt.plot(predicted_dict['f_val1'], color='grey', linestyle='--')
#plt.plot(predicted_dict['f_val2'], color='black', linestyle='--')
#plt.plot(fact_dict['outgoing_deg'], color='grey')
#plt.plot(fact_dict['ingoing_deg'], color='black')
##plt.legend(['Исходящие степени (план)', 'Входящие степени (план)', 'Исходящие степени (факт)', 'Входящие степени (факт)'])

alpha1: 1.1780972450961724
alpha2: 0.39269908169872414
betta1: -1414.7291475506336
betta_temp: 0
x: 0
alpha2: 0.39269908169872414
betta2: 0

random_swap: 1
swap_counter: 207262
outgoing_error: 35384
ingoing_error: 34801

f_val1: 206774
f_val2: 206403
number of edges: 206774


# Эксперименты

In [13]:
from operator import itemgetter

def reject_outliers(data_x, data_y, coeff):
    data = []
    for i in range(len(data_x)):
        data.append((data_x[i],data_y[i]))
    data.sort(key=itemgetter(1), reverse=True)
    #print(data)
    #"""
    for i in range(len(data_x)):
        if not (data[i][1] / data[i+1][1] > coeff):
            _data = data[i:]
            _data_x = []
            _data_y = []
            for j in range(len(_data)):
                _data_x.append(_data[j][0])
                _data_y.append(_data[j][1])
            return _data_x, _data_y
    #"""

In [15]:
%%time
graphs_dict = {}

num_of_nodes = 100

def get_edges_density(num_of_nodes, num_of_edges):
    return num_of_edges / (num_of_nodes * (num_of_nodes - 1))

def get_area_density(error_area, under_curve_area):
    return error_area / under_curve_area
"""
alpha1_list = []
alpha2_list = []
betta1_list = []
betta2_list = []

out_error_list = []
in_error_list = []
swap_counter_list = []
num_of_edges_list = []

edge_density_list = []
area_density_list = []
area_outgoing_density_list = []
area_ingoing_density_list = []

alpha_check_list = []
"""
#num_of_nodes_list = [50, 100, 150, 200, 250]
num_of_nodes_list = [100]
#alpha1_predefined_values = [-10]
num_of_experiments = 5000

#for alpha1 in alpha1_predefined_values:

alpha1 = -1000   # 1
#alpha1 = 5 * math.pi / 12

alpha2 = -1000
#alpha2 = 5 * math.pi / 12

betta1 = -1000            # 1
betta1_packet = []
#betta1 = 0
#betta1_packet = [0, 0]

print('alpha1:', alpha1)
print('betta1:', betta1)
for num_of_nodes in num_of_nodes_list:
    print('Number of nodes:', num_of_nodes)
    for i in range(num_of_experiments):
        if i % 500 == 0:
            print(i)
        
        while 1==1:
            predicted_dict = get_predicted(num_of_nodes, alpha1, alpha2, betta1, betta1_packet)
            fact_dict = get_fact(predicted_dict, num_of_nodes)

            graphs_dict[i] = {}
            graphs_dict[i]['predicted'] = predicted_dict
            graphs_dict[i]['fact'] = fact_dict
            
            if fact_dict['G'].number_of_nodes() == num_of_nodes:
                break
        """
        #error_list.append(fact_dict['outgoing_error'] + fact_dict['ingoing_error'])
        alpha1_list.append(predicted_dict['alpha1'])
        alpha2_list.append(predicted_dict['alpha2'])
        betta1_list.append(predicted_dict['betta1'])
        betta2_list.append(predicted_dict['betta2'])
        in_error_list.append(fact_dict['ingoing_error'])
        out_error_list.append(fact_dict['outgoing_error'])
        num_of_edges_list.append(fact_dict['num_of_edges'])
        swap_counter_list.append(fact_dict['swap_counter'])
        
        if (predicted_dict['alpha1'] >= (math.pi / 4)) and (predicted_dict['alpha2'] >= (math.pi / 4)):
            alpha_check_list.append(1)
        elif (predicted_dict['alpha1'] < (math.pi / 4)) and (predicted_dict['alpha2'] >= (math.pi / 4)):
            alpha_check_list.append(2)
        elif (predicted_dict['alpha1'] < (math.pi / 4)) and (predicted_dict['alpha2'] < (math.pi / 4)):
            alpha_check_list.append(3)
        elif (predicted_dict['alpha1'] >= (math.pi / 4)) and (predicted_dict['alpha2'] < (math.pi / 4)):
            alpha_check_list.append(4)

        edge_density_list.append(get_edges_density(num_of_nodes, fact_dict['num_of_edges']))
        area_density_list.append(get_area_density(fact_dict['ingoing_error'] + fact_dict['outgoing_error'],\
                                                  fact_dict['num_of_edges']))
        area_outgoing_density_list.append(get_area_density(fact_dict['outgoing_error'], fact_dict['num_of_edges']))
        area_ingoing_density_list.append(get_area_density(fact_dict['ingoing_error'], fact_dict['num_of_edges']))
        """
        #print(fact_dict['ingoing_error'], fact_dict['outgoing_error'], sum(predicted_dict['f_val1']), sum(predicted_dict['f_val2']), fact_dict['num_of_edges'])
print('Done!')

alpha1: -1000
betta1: -1000
Number of nodes: 100
0
500
1000
1500
2000
2500
3000
3500
4000
4500
Done!
Wall time: 13min 37s


In [14]:
#len(graphs_dict)

In [15]:
#with open('...', 'wb') as f:
#    pickle.dump(graphs_dict, f)